# Running GLM for MCSE task

This script runs a simple glm on one subject for the MCSE condition
and returns the contrasts obtained.
Then it applies the same thing to all subjects, and eventually performs a one-sample test to detect effects at the population level.

This task (described in [https://doi.org/10.1523/JNEUROSCI.6048-11.2012]) was originally used to study whether visual search processes of a salient target can be thought as a purely bottom-up process, or if it requires action from top-down attentional processes.

The task consisted in the presentation of an array of 35 “L” letters, rotated at different angles, together with a target “T” letter (total 36 stimuli in each trial). Subjects were instructed to search for the target and indicate whether it was on the left or right side of the grid, by pressing respectively with the index or middle finger on a 5-button response box. There were two conditions: high-salience (the target is gray while the other stimuli is black) and low-salience (all stimuli are gray).

The two conditions were presented alternatively in blocks, with 6 blocks of 10 trials
each. Each trial was presented for 3 s with an inter-stimulus interval of 1 s. There was
also a 20 s fixation cross between blocks. Data were acquired in two separated runs, one in the "Posterior to Anterior" (PA) phase encoding direction, one in the "Anterior to Posterior" phase encoding direction.

In [ ]:
%matplotlib inline

import glob
import numpy as np
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / "scripts"))

## Explore the data and understand them

In [ ]:
# This is a grey matter mask, that defines which regions of the brain 
from nilearn.plotting import plot_roi
gm_mask = 'data/gm_mask_3mm.nii.gz'
plot_roi(gm_mask)

In [ ]:
# This will download the data or locate it if already downloaded
from download_data import download_data
import os
download_data()
data_dir = 'data/3mm'
subjects = [os.path.basename(x) for x in
            glob.glob(os.path.join(data_dir, 'sub-*'))]
print(subjects)

In [ ]:
# Time of repetition, i.e. time between consecutive scans (in seconds)
import json
TR = json.load(open('data/3mm/task-MCSE_dir-ap_bold.json'))['RepetitionTime']
print(TR)

In [ ]:
# Create a results directory, where we will generate all ouputs for this session
# Can be any valid path on disk
write_dir = 'results'
if not os.path.exists(write_dir):
    os.mkdir(write_dir)

Next we define a utility function to fetch individual data

In [ ]:
def fetch_data(data_dir, subject):
    """Helper to get all the necessary data from the downloaded material.
    
    Parameters
    ----------
    data_dir: string,
              Path to the main data dirctory
              
    subject: string,
             Subject identifier
    
    Returns
    -------
    brain image: list of strings,
                 4D brain images for the subject
                 
    confound: list os strings,
              Confound files for the subject
              
    event: list of strings,
           List of event files for the subject
    """
    # prepare the data for the subject
    bold = []
    confounds = []
    events = []
    # infer the session name for the data; 
    # we explicitly don't want to take the session names 'ses-00'
    session = [os.path.basename(x) for x in glob.glob(
        os.path.join(data_dir, subject, 'ses-*')) if 'ses-00' not in x][0]
    func_dir = os.path.join(data_dir, subject, session, 'func')
    for direction in ['pa', 'ap']: # 'pa' and 'ap'  correspond to 2 runs
        # fMRI data
        wc = os.path.join(
            func_dir,
            '{}_{}_task-MCSE_dir-{}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'.format(
                subject, session, direction))
        bold.append(wc)
        # associated confound data (motion, CompCor output)
        wc = os.path.join(
            func_dir,
            '{}_{}_task-MCSE_dir-{}_desc-confounds_timeseries.tsv'.format(
                subject, session, direction))
        confounds.append(wc)
        # associated event descriptors
        wc = os.path.join(func_dir,
            '{}_{}_task-MCSE_dir-{}_events.tsv'.format(subject, session, direction))
        events.append(wc)
    return bold, confounds, events

In [ ]:
# Get the data for one subject
subject = subjects[0]
bold, confounds, events = fetch_data(data_dir, subject)

In [ ]:
# Explore the experimental design of the experiment
import pandas as pd
from nilearn.plotting import plot_event, show
event_dfs = [pd.read_csv(event, sep='\t') for event in events]
plot_event(event_dfs, figsize=(9, 4))
show()

There are 5 types of events: low and high salience stimuli, presented either on the left or right side of the display.
There is also a dummy BFix condition ("fixation"), that is of no interest and actually corresponds to a baseline. We may want to get rid of it. 

In [ ]:
# GLM specification: specify repetition time, brain mask, 
# spatial and temporal filtering parameters
from nilearn.glm.first_level import FirstLevelModel
model = FirstLevelModel(
    t_r=TR,            # provide repetition time
    mask_img=gm_mask,  # restrict analysis to grey matter regions 
    smoothing_fwhm=5,  # in mm
    high_pass = 0.008, # in hz
    hrf_model='spm + derivative', # hemodynamic model to use: SPM model and its time derivative
    slice_time_ref=.5  # The design will be sampled at the middle of each volume acquisition
)

In [ ]:
# utility function to specify contrasts
def mcse_contrasts():
    """Specification of relevant MCSE contrasts"""
    # to make matters simple, we specify only two contrasts
    # 'low-high salience' addresses the difference in brain activity btw the low and high salience condition. 
    # Obviously, low salience involves higher spatial attention, all other things being equal.
    # salience_left-right' considers whether the target is on the left or right side of the display
    contrasts = {
        'low-high salience': 'low_salience_left + low_salience_right' +\
            '- hi_salience_left - hi_salience_right',
        'salience_left-right': 'low_salience_left - low_salience_right' +\
            '+ hi_salience_left - hi_salience_right'
    }
    return contrasts

contrasts = mcse_contrasts()

## Run the analysis in one subject

In [ ]:
# Prepare write dir
subject_dir = os.path.join(write_dir, subject)
if not os.path.exists(subject_dir):
    os.mkdir(subject_dir)
    
event_dfs = []
# loop on the two acquisitions for this subject
for i, (event, confound, img) in enumerate(zip(events, confounds, bold)):
    # Slightly modify the event files not to model implicit baseline, called "Bfix"
    event_df = pd.read_csv(event, index_col=None, sep='\t')
    event_df = event_df[event_df. trial_type != 'Bfix']
    event_dfs.append(event_df) 

    # fit the model
    model.fit(img, events=event_df, confounds=confound)
    # write the output as a report (HTML page)
    report = model.generate_report(contrasts)
    report.save_as_html(
        os.path.join(subject_dir, 'report_{}.html'.format(i)))

In [ ]:
# just take a look at the events that were actually used
plot_event(event_dfs, figsize=(9, 4))
show()

OK, but we need to have one stat map that summarizes the effects per subject. To achieve this, directly compute a fixed effects model on the data.

In [ ]:
# for this, provide list of fMRI data, events and confound descriptors
# the report displays the fixed effects for the specified contrasts
model.fit(bold, events=event_dfs, confounds=confounds)
report = model.generate_report(contrasts)
# the report can be written on disk, or displayed, or both !
report.save_as_html(os.path.join(subject_dir, 'report_ffx.html'))
report

Save contrast images to disk for future use in group analysis

In [ ]:
for  key in contrasts.keys():
    stat_img = model.compute_contrast(contrasts[key], output_type='z_score')
    output_file = os.path.join(subject_dir, f'{key}_z_score.nii.gz')
    stat_img.to_filename(output_file)

## Run the analysis on all subjects

Great, we did it on one subject. Let's write a loop to run all that stuff on all subjects of the study. We just do the minimal thing: 

In [ ]:
for subject in subjects:
    subject_dir = os.path.join(write_dir, subject)
    if not os.path.exists(subject_dir):
        os.mkdir(subject_dir)
    
    bold, confounds, events = fetch_data(data_dir, subject)
    event_dfs = []
    for event in events:
        event_df = pd.read_csv(event, index_col=None, sep='\t')
        event_dfs.append(event_df[event_df. trial_type != 'Bfix'])
    
    model.fit(bold, events=event_dfs, confounds=confounds)
    for  key in contrasts.keys():
        stat_img = model.compute_contrast(
                contrasts[key], output_type='z_score')
        output_file = os.path.join(subject_dir, f'{key}_z_score.nii.gz')
        stat_img.to_filename(output_file)

## Group analysis (one-sample test)

Do a one-sample test across subjects to discover which brain regions display a positive effect across individuals.

In [ ]:
from nilearn.glm.second_level import SecondLevelModel
# we create a trivial design matrix with only an intercept
# since we only study the average effect of the contrast across subjects
design_matrix = pd.DataFrame([1] * len(subjects), columns=['intercept'])
second_level_model = SecondLevelModel(mask_img=gm_mask)
alpha = .05 # desired significance level for FDR control

In [ ]:
from nilearn.glm import threshold_stats_img
from nilearn.plotting import view_img
views = {}
for key in contrasts.keys():
    # Get the images
    imgs = [os.path.join(write_dir, subject, f'{key}_z_score.nii.gz')
            for subject in subjects]    
    second_level_model.fit(imgs, design_matrix=design_matrix)
    z_map = second_level_model.compute_contrast(output_type='stat')
    
    # Take a statistical threshold, corresponding to fdr <= alpha
    _, threshold = threshold_stats_img(
        z_map, alpha=alpha, height_control='fdr')
    print(key, threshold)
    
    # look at the result
    views[key] = view_img(
        z_map, threshold=threshold, title=f'{key}, fdr <{alpha}')

In [ ]:
views['salience_left-right']

We mostly see differences in the early (retinotopic) cortex, where subjects were looking at.

In [ ]:
views['low-high salience']

Here we see the effect of salience on brain activity across participants.

In [ ]:
# Save the map for the 'low-high salience' contrast
key = 'low-high salience'
imgs = [os.path.join(write_dir, subject, f'{key}_z_score.nii.gz')
            for subject in subjects]
second_level_model.fit(imgs, design_matrix=design_matrix)
filename = os.path.join(write_dir, f'group_{key}_z_score.nii.gz')
second_level_model.compute_contrast(output_type='stat').to_filename(filename)